# *Сеть персептронов*

#### 1. Установка библиотек

In [194]:
!pip install numpy
!pip install matplotlib

#### 2. Импорт библиотек

In [195]:
import numpy as np
import random
import matplotlib.pyplot as plt

#### 3. Шаблон нейросети

In [196]:
class Perceptron:
    def __init__(self,input_data:np.array, output_data, learning_rate):
        self.x_train = input_data
        self.y_train = output_data
        self.w = np.random.uniform(size=(1,x_train.shape[1]))
        self.lr = learning_rate
        self.index_list = list(range(self.x_train.shape[0]))
        
    def __show_info(self,x,y,p_out):
        outprint_W = ""
        for i in range(len(self.w)):
            for j in range(len(self.w[i])):
                outprint_W += (f"w{j} ={self.w[i][j]:5.2f} ")
        print(f"""Weights: {outprint_W}
Input Data:  {x}
Expected output {y}  Predicated Output: {p_out}""")
    
    def sigmoid(self, x):
        return 1/(1+np.exp(-x))
    
    def relu(self, x):
        return np.maximum(0, x)
    
    def step_function(self,x):
        if x < 0: return -1
        else: return 1
    
    def train(self):
        all_correct = False
        while not all_correct:
            all_correct = True
            random.shuffle(self.index_list)

            for i in self.index_list:
                x = self.x_train[i]
                y = self.y_train[0][i]
                z = np.dot(self.w, x)

                p_out = self.step_function(z)
                if  y != p_out: # Обновить веса, когда неправильно
                    print("____________________________\n 🔁 Updating the weights 🔁")
                    for j in range(0, len(self.w[0])):
                        self.w[0][j] += (y * self.lr * x[j])
                    all_correct = False
                    self.__show_info(x,y,p_out) # Показать обновлённые веса
        print("____________________________\n ✅ Weights updated ✅")
        self.__show_info(x,y,p_out)
                    
    def predict(self, input_data):
        for i in range(len(input_data)):
            z = np.dot(self.w, input_data[i])
            p_out = self.step_function(z)
            
        return p_out
        

#### 4. Константы и тренировочные данные

In [197]:
LEARNING_RATE = 0.01
CURRENT_HIDDEN = 1
CURRENT_OUTPUT = 2


x_train = np.array([(1.0,-1.0,-1.0),
           (1.0,-1.0,1.0),
           (1.0,1.0,-1.0),
           (1.0,1.0,1.0)])
y_AND_train = np.array([[-1.0,-1.0,-1.0,1.0]])


print(x_train)


[[ 1. -1. -1.]
 [ 1. -1.  1.]
 [ 1.  1. -1.]
 [ 1.  1.  1.]]


In [198]:
nand_perceptron = Perceptron(x_train, np.array([[1.0,1.0,1.0,-1.0]]), LEARNING_RATE)
or_perceptron = Perceptron(x_train, np.array([[-1.0,1.0,1.0,1.0]]), LEARNING_RATE)
and_perceptron = Perceptron(x_train, np.array([[-1.0,-1.0,-1.0,1.0]]), LEARNING_RATE)

In [199]:
nand_perceptron.train()

____________________________
 🔁 Updating the weights 🔁
Weights: w0 = 0.81 w1 = 0.60 w2 = 0.34 
Input Data:  [1. 1. 1.]
Expected output -1.0  Predicated Output: 1
____________________________
 🔁 Updating the weights 🔁
Weights: w0 = 0.82 w1 = 0.59 w2 = 0.33 
Input Data:  [ 1. -1. -1.]
Expected output 1.0  Predicated Output: -1
____________________________
 🔁 Updating the weights 🔁
Weights: w0 = 0.83 w1 = 0.58 w2 = 0.32 
Input Data:  [ 1. -1. -1.]
Expected output 1.0  Predicated Output: -1
____________________________
 🔁 Updating the weights 🔁
Weights: w0 = 0.82 w1 = 0.57 w2 = 0.31 
Input Data:  [1. 1. 1.]
Expected output -1.0  Predicated Output: 1
____________________________
 🔁 Updating the weights 🔁
Weights: w0 = 0.83 w1 = 0.56 w2 = 0.30 
Input Data:  [ 1. -1. -1.]
Expected output 1.0  Predicated Output: -1
____________________________
 🔁 Updating the weights 🔁
Weights: w0 = 0.82 w1 = 0.55 w2 = 0.29 
Input Data:  [1. 1. 1.]
Expected output -1.0  Predicated Output: 1
___________________

In [200]:
and_perceptron.train()

____________________________
 🔁 Updating the weights 🔁
Weights: w0 = 0.49 w1 = 0.91 w2 = 0.17 
Input Data:  [ 1.  1. -1.]
Expected output -1.0  Predicated Output: 1
____________________________
 🔁 Updating the weights 🔁
Weights: w0 = 0.48 w1 = 0.90 w2 = 0.18 
Input Data:  [ 1.  1. -1.]
Expected output -1.0  Predicated Output: 1
____________________________
 🔁 Updating the weights 🔁
Weights: w0 = 0.47 w1 = 0.89 w2 = 0.19 
Input Data:  [ 1.  1. -1.]
Expected output -1.0  Predicated Output: 1
____________________________
 🔁 Updating the weights 🔁
Weights: w0 = 0.46 w1 = 0.88 w2 = 0.20 
Input Data:  [ 1.  1. -1.]
Expected output -1.0  Predicated Output: 1
____________________________
 🔁 Updating the weights 🔁
Weights: w0 = 0.45 w1 = 0.87 w2 = 0.21 
Input Data:  [ 1.  1. -1.]
Expected output -1.0  Predicated Output: 1
____________________________
 🔁 Updating the weights 🔁
Weights: w0 = 0.44 w1 = 0.86 w2 = 0.22 
Input Data:  [ 1.  1. -1.]
Expected output -1.0  Predicated Output: 1
__________

In [201]:
or_perceptron.train()

____________________________
 🔁 Updating the weights 🔁
Weights: w0 = 0.82 w1 = 0.92 w2 = 0.05 
Input Data:  [ 1. -1.  1.]
Expected output 1.0  Predicated Output: -1
____________________________
 🔁 Updating the weights 🔁
Weights: w0 = 0.83 w1 = 0.91 w2 = 0.06 
Input Data:  [ 1. -1.  1.]
Expected output 1.0  Predicated Output: -1
____________________________
 🔁 Updating the weights 🔁
Weights: w0 = 0.84 w1 = 0.90 w2 = 0.07 
Input Data:  [ 1. -1.  1.]
Expected output 1.0  Predicated Output: -1
____________________________
 🔁 Updating the weights 🔁
Weights: w0 = 0.85 w1 = 0.89 w2 = 0.08 
Input Data:  [ 1. -1.  1.]
Expected output 1.0  Predicated Output: -1
____________________________
 ✅ Weights updated ✅
Weights: w0 = 0.85 w1 = 0.89 w2 = 0.08 
Input Data:  [ 1.  1. -1.]
Expected output 1.0  Predicated Output: 1


In [217]:
x_test = np.array([(1.0,1.0,-1.0)])
print(x_test)

xor = and_perceptron.predict(np.array([(1.0, nand_perceptron.predict(x_test), or_perceptron.predict(x_test))]))
print(xor)

[[ 1.  1. -1.]]
1
